# XSD to CSV Conversion Testing

### Import needed libraries

- `zipfile` – for working with ZIP archives
- `xmlschema` – to read and validate XSD files (`pip install xmlschema`)
- `pandas` – for working with tables and dataframes (`pip install pandas`)
- `lxml.etree` – to parse and read XML/XSD structure (`pip install lxml`)
- `pathlib.Path` – for path handling

Make sure to install missing ones using `pip` if needed.

In [1]:
import zipfile
import xmlschema
import pandas as pd
from lxml import etree
from pathlib import Path

### Set paths for input ZIP and output folder

This cell sets:
- `input_zip_path`: path to the input ZIP file with XSDs
- `output_path`: where to extract files

If the folder doesn't exist, it will be created.

In [2]:
# Define the path to the input zip file and output directory
input_zip_path = Path("../tests/data/JVF_DTM_143_XSD.zip")
output_path = Path("../tests/output/JVF_DTM_143_XSD")

# Create the output directory if it doesn't exist
output_path.mkdir(parents=True, exist_ok=True)

### Unzip the input file

Extracts the ZIP archive to the output folder and prints the extracted file names.

In [3]:
# Extract the zip file
with zipfile.ZipFile(input_zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_path)

# Show extracted files
print(f"Extracted to: {output_path}")
print(sorted(f.name for f in output_path.iterdir()))

Extracted to: ..\tests\output\JVF_DTM_143_XSD
['str1.csv', 'str2.csv', 'xsd']


### Load and parse XSD files

Defines a function that finds all `.xsd` files in the folder, parses them into XML trees, and returns a list of (file name, XML root element) pairs.

In [4]:
# Set the path to the folder with extracted XSD files
xsd_dir = output_path / "xsd"

def load_xsd_files(directory):
    """Load all XSD files from the given directory and parse them into XML trees.
    
    Args:
        directory (Path): Base directory to search for .xsd files.

    Returns:
        list of (filename, XML root element) tuples.
    """
    
    # Initialize an empty list to store parsed XSD file information
    xsd_files = []
    
    # Recursively find all .xsd files
    for path in directory.rglob("*.xsd"):
        try:
            # Parse the file into an XML tree
            tree = etree.parse(str(path))
    
            # Add the file name and root element to the list
            xsd_files.append((path.name, tree.getroot()))
    
        except etree.XMLSyntaxError as e:
            # Handle invalid XML syntax
            print(f"[XMLSyntaxError] Skipping {path.name}: {e}")
    
        except Exception as e:
            #  Handle all unexpected errors
            print(f"[UnexpectedError] Could not process {path.name}: {e}")
            raise  # Re-raise the exception

    return xsd_files

# Call the function to load and parse the XSD files
xsd_files = load_xsd_files(xsd_dir)

### Create summary table of XSD files

Builds a simple table showing each XSD file's name, root tag, and number of elements.

In [5]:
# Initialize empty lists for storing data
summary = []

for file_name, root in xsd_files:
    summary.append({
        "File Name": file_name,
        "Root Tag": root.tag,
        "Number of Elements": len(root)
    })

df_summary = pd.DataFrame(summary)
df_summary

,File Name,Root Tag,Number of Elements
0,atributy.xsd,{http://www.w3.org/2001/XMLSchema}schema,185
1,common.xsd,{http://www.w3.org/2001/XMLSchema}schema,4
2,doprovodne_informace.xsd,{http://www.w3.org/2001/XMLSchema}schema,29
3,extenze.xsd,{http://www.w3.org/2001/XMLSchema}schema,1
4,servis.xsd,{http://www.w3.org/2001/XMLSchema}schema,1
...,...,...,...
442,spatialReferencing.xsd,{http://www.w3.org/2001/XMLSchema}schema,14
443,geometry.xsd,{http://www.w3.org/2001/XMLSchema}schema,19
444,gss.xsd,{http://www.w3.org/2001/XMLSchema}schema,7
445,gts.xsd,{http://www.w3.org/2001/XMLSchema}schema,7


In [11]:
# Path to the folder with extracted XSD files
xsd_objects_path = output_path / "xsd" / "objekty"

records = []
seen_global = set()  # Track all (filename, nazev) pairs to avoid duplicates

# Loop through all .xsd files in the directory
for file_path in xsd_objects_path.glob("*.xsd"):
    try:
        # Parse the XSD file into an XML tree
        tree = etree.parse(str(file_path))
        root = tree.getroot()

        # Find all <complexType> elements in the schema
        complex_types = root.xpath(".//*[local-name()='complexType']")

        for complex_type in complex_types:
            # Find nested <element> definitions
            for element in complex_type.xpath(".//*[local-name()='element']"):
                ref = element.get("ref")
                min_occurs = element.get("minOccurs")

                if ref:
                    key = (file_path.name, ref)

                    # If it's a gml: reference and not already added
                    if ref.startswith("gml:") and key not in seen_global:
                        seen_global.add(key)
                        records.append({
                            "filename": file_path.name,
                            "nazev": str([ref]),  # Format as list string e.g. ['gml:...']
                            "minOccurs": "0"
                        })

                    # If it's an atr: reference and not already added
                    elif ref.startswith("atr:") and key not in seen_global:
                        seen_global.add(key)
                        records.append({
                            "filename": file_path.name,
                            "nazev": ref,
                            "minOccurs": min_occurs  # May be None
                        })

    except etree.XMLSyntaxError as e:
        print(f"[XMLSyntaxError] {file_path.name}: {e}")
    except Exception as e:
        print(f"[Error] Failed to process {file_path.name}: {e}")
        raise

# Create DataFrame from extracted records
df_str2 = pd.DataFrame(records)

# Save the DataFrame to CSV
df_str2.to_csv(output_path / "str2.csv", index=False)

# Display the resulting DataFrame
df_str2.head(30)

,filename,nazev,minOccurs
0,BP_plynovodni_site-plocha.xsd,atr:SpolecneAtributyVsechObjektu,None
1,BP_plynovodni_site-plocha.xsd,atr:SpolecneAtributyObjektuPasemTI,None
2,BP_plynovodni_site-plocha.xsd,['gml:surfaceProperty'],0
3,BP_plynovodni_site-plocha.xsd,atr:OblastObjektuKI,0
4,BP_podzemniho_zasobniku_plynu-plocha.xsd,atr:SpolecneAtributyVsechObjektu,None
5,BP_podzemniho_zasobniku_plynu-plocha.xsd,atr:SpolecneAtributyObjektuPasemTI,None
6,BP_podzemniho_zasobniku_plynu-plocha.xsd,atr:TypPodzemnihoZasobnikuPlynu,None
7,BP_podzemniho_zasobniku_plynu-plocha.xsd,['gml:surfaceProperty'],0
8,BP_podzemniho_zasobniku_plynu-plocha.xsd,atr:OblastObjektuKI,0
9,BP_zarizeni_PKO-plocha.xsd,atr:SpolecneAtributyVsechObjektu,None


## str1

* So far the column zaznamy is missing,
* mby instead of using xpath we could try iter() and find()


In [7]:
# Path to the folder with extracted .xsd files
xsd_objects_path = output_path / "xsd" / "objekty"

# Initialize an empty list for the result records
records = []

# Iterate over all .xsd files in the folder
for file_path in xsd_objects_path.glob("*.xsd"):
    try:
        # Parse the XSD file into an XML tree
        tree = etree.parse(str(file_path))
        root = tree.getroot()

        # Extract the target namespace of the schema (if present)
        namespace = root.attrib.get("targetNamespace", "")
        
        # Initialize default placeholder values
        element_name = ""
        element_type = ""
        code_base_fixed = ""
        code_base_use = ""
        code_suffix_fixed = ""
        code_suffix_use = ""
        kategorie_objektu = ""
        skupina_objektu = ""
        obsahova_cast = ""
        geom_minOccurs = ""
        geom = ""
        oblastObjektuKI = 0
        ki_minOccurs = ""

        # Find all complexType definitions
        complex_types = root.xpath(".//*[local-name()='complexType']")

        for complex_type in complex_types:
            # Search for nested element definitions within each complexType
            for element in complex_type.xpath(".//*[local-name()='element']"):
                name = element.get("name")
                ref = element.get("ref")
                min_occurs = element.get("minOccurs")

                # Locate the first element whose type starts with this namespace
                first_element = root.xpath(
                    f".//*[local-name()='element'][starts-with(@type, '{namespace}:')]"
                )
                if first_element:
                    element_name = first_element[0].get("name")
                    element_type = first_element[0].get("type")

                # If this element is ObjektovyTypNazev, extract its attributes
                if name == "ObjektovyTypNazev":
                    code_base_fixed_list = element.xpath(
                        ".//*[local-name()='attribute' and @name='code_base']/@fixed"
                    )
                    code_base_use_list = element.xpath(
                        ".//*[local-name()='attribute' and @name='code_base']/@use"
                    )
                    code_suffix_fixed_list = element.xpath(
                        ".//*[local-name()='attribute' and @name='code_suffix']/@fixed"
                    )
                    code_suffix_use_list = element.xpath(
                        ".//*[local-name()='attribute' and @name='code_suffix']/@use"
                    )

                    if code_base_fixed_list:
                        code_base_fixed = int(code_base_fixed_list[0])
                    if code_base_use_list:
                        code_base_use = code_base_use_list[0]
                    if code_suffix_fixed_list:
                        code_suffix_fixed = int(code_suffix_fixed_list[0])
                    if code_suffix_use_list:
                        code_suffix_use = code_suffix_use_list[0]

                # If this element is KategorieObjektu, record its fixed value
                elif name == "KategorieObjektu":
                    kategorie_objektu = element.get("fixed", "")
                # If this element is SkupinaObjektu, record its fixed value
                elif name == "SkupinaObjektu":
                    skupina_objektu = element.get("fixed", "")
                # If this element is ObsahovaCast, record its fixed value
                elif name == "ObsahovaCast":
                    obsahova_cast = element.get("fixed", "")

                # If this element is GeometrieObjektu, record reference and minOccurs
                elif name == "GeometrieObjektu":
                    geom_minOccurs = min_occurs
                    ref_elem = element.xpath(
                        ".//*[local-name()='element' and @ref]"
                    )
                    geom = ref_elem[0].get("ref") if ref_elem else ""

                # If the element references atr:OblastObjektuKI, set the flag
                if ref == "atr:OblastObjektuKI":
                    ki_minOccurs = min_occurs
                    oblastObjektuKI = 1

        # Append the collected data for this XSD file to records
        records.append({
            "filename": file_path.name,
            "namespace": namespace,
            "zaznamy": 'XXX',
            "geom":geom,
            "OblastObjektuKI":oblastObjektuKI,
            "name":element_name,
            "type":element_type,
            "code_base_fixed":code_base_fixed,
            "code_base_use":code_base_use,
            "code_suffix_fixed":code_suffix_fixed,
            "code_suffix_use":code_suffix_use,
            "KategorieObjektu":kategorie_objektu,
            "SkupinaObjektu":skupina_objektu,
            "ObsahovaCast":obsahova_cast,
            "geom_minOccurs":geom_minOccurs,
            "oblaskKI_minOccurs":ki_minOccurs
        })

    except etree.XMLSyntaxError as e:
        print(f"[XMLSyntaxError] {file_path.name}: {e}")
    except Exception as e:
        print(f"[Error] Failed to process {file_path.name}: {e}")
        raise

# Create DataFrame from collected records
df_str1 = pd.DataFrame(records)

# Save DataFrame to CSV file
output_csv_path = output_path / "str1.csv"
output_csv_path.parent.mkdir(parents=True, exist_ok=True)
df_str1.to_csv(output_csv_path, index=False)

# Show the DataFrame
df_str1

,filename,namespace,zaznamy,geom,OblastObjektuKI,name,type,code_base_fixed,code_base_use,code_suffix_fixed,code_suffix_use,KategorieObjektu,SkupinaObjektu,ObsahovaCast,geom_minOccurs,oblaskKI_minOccurs
0,BP_plynovodni_site-plocha.xsd,bpplsi,XXX,gml:surfaceProperty,1,BPPlynovodniSite,bpplsi:BPPlynovodniSiteType,100000290,required,3,required,Ochranná a bezpečnostní pásma,Ochranné a bezpečnostní pásmo,TI,0,0
1,BP_podzemniho_zasobniku_plynu-plocha.xsd,bpppol,XXX,gml:surfaceProperty,1,BPPodzemnihoZasobnikuPlynu,bpppol:BPPodzemnihoZasobnikuPlynuType,100000369,required,3,required,Ochranná a bezpečnostní pásma,Ochranné a bezpečnostní pásmo,TI,0,0
2,BP_zarizeni_PKO-plocha.xsd,bpzpko,XXX,gml:surfaceProperty,1,BPZarizeniPKO,bpzpko:BPZarizeniPKOType,100000291,required,3,required,Ochranná a bezpečnostní pásma,Ochranné a bezpečnostní pásmo,TI,0,0
3,budova-defbod.xsd,buddef,XXX,gml:pointProperty,0,BudovaDefinicniBod,buddef:BudovaDefinicniBodType,100000001,required,4,required,Budovy,Objekt budovy,ZPS,None,
4,budova-plocha.xsd,budpol,XXX,gml:surfaceProperty,0,BudovaPlocha,budpol:BudovaPlochaType,100000001,required,3,required,Budovy,Objekt budovy,ZPS,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,zed-linie.xsd,zedlin,XXX,gml:curveProperty,0,ZedLinie,zedlin:ZedLinieType,100000168,required,2,required,Součásti a příslušenství staveb,Stavba společná pro více skupin,ZPS,None,
354,zed-plocha.xsd,zedpol,XXX,gml:surfaceProperty,0,ZedPlocha,zedpol:ZedPlochaType,100000168,required,3,required,Součásti a příslušenství staveb,Stavba společná pro více skupin,ZPS,None,
355,zeleznicni_prejezd-plocha.xsd,zprpol,XXX,gml:surfaceProperty,0,ZeleznicniPrejezd,zprpol:ZeleznicniPrejezdType,100000022,required,3,required,Dopravní stavby,Drážní doprava,DI,None,
356,zemedelska_plocha-defbod.xsd,zepdef,XXX,gml:pointProperty,0,ZemedelskaPlochaDefinicniBod,zepdef:ZemedelskaPlochaDefinicniBodType,100000207,required,4,required,"Vodstvo, vegetace a terén",Hospodářská plocha,ZPS,None,


In [8]:
# # Helper function to extract the targetNamespace from an XSD file
# def get_target_namespace(file_path):
#     try:
#         tree = etree.parse(file_path)
#         root = tree.getroot()
#         return root.attrib.get("targetNamespace", "")
#     except Exception as e:
#         print(f"Error while reading {file_path}: {e}")
#         return ""
# 
# # Build a list of dictionaries with filenames and their associated namespaces
# xsd_info = []
# for fname in os.listdir(xsd_objekty_path):
#     if fname.endswith(".xsd"):
#         full_path = os.path.join(xsd_objekty_path, fname)
#         namespace = get_target_namespace(full_path)
#         xsd_info.append({"filename": fname, "namespace": namespace})
# 
# # Convert the list into a pandas DataFrame
# df_xsd_files = pd.DataFrame(xsd_info)
# 
# # Show the DataFrame
# df_xsd_files

In [9]:
from pathlib import Path
from lxml import etree
import pandas as pd

output_path = Path(r"C:\Users\michal\Desktop\FGIS\2025-b-jvf-dtm-xsd\tests\output\JVF_DTM_143_XSD")
xsd_objects_path = output_path / "xsd" / "objekty"

records = []

for file_path in xsd_objects_path.glob("*.xsd"):
    try:
        tree = etree.parse(str(file_path))
        root = tree.getroot()
        namespace = root.attrib.get("targetNamespace", "")

        record = {
            "filename": file_path.name,
            "namespace": namespace,
        }

        first_element = root.xpath(f".//*[local-name()='element'][starts-with(@type, '{namespace}:')]")
        if first_element:
            record["name"] = first_element[0].get("name", "")
            record["type"] = first_element[0].get("type", "")

        for complex_type in root.xpath(".//*[local-name()='complexType']"):
            for element in complex_type.xpath(".//*[local-name()='element']"):
                name = element.get("name")
                ref = element.get("ref")
                typ = element.get("type")
                min_occurs = element.get("minOccurs")

                if name:
                    record[f"{name}_minOccurs"] = min_occurs or ""
                    fixed = element.get("fixed")
                    if fixed:
                        record[name] = fixed

                if name == "ObjektovyTypNazev":
                    for attr in ["code_base", "code_suffix"]:
                        fixed_list = element.xpath(f".//*[local-name()='attribute' and @name='{attr}']/@fixed")
                        use_list = element.xpath(f".//*[local-name()='attribute' and @name='{attr}']/@use")
                        if fixed_list:
                            record[f"{attr}_fixed"] = fixed_list[0]
                        if use_list:
                            record[f"{attr}_use"] = use_list[0]

                if name == "GeometrieObjektu":
                    record["geom_minOccurs"] = min_occurs or ""
                    ref_elem = element.xpath(".//*[local-name()='element' and @ref]")
                    if ref_elem:
                        record["geom"] = ref_elem[0].get("ref", "")

                if ref == "atr:OblastObjektuKI":
                    record["oblaskKI_minOccurs"] = min_occurs or ""
                    record["OblastObjektuKI"] = 1

        records.append(record)

    except etree.XMLSyntaxError as e:
        print(f"[XMLSyntaxError] {file_path.name}: {e}")
    except Exception as e:
        print(f"[Error] Failed to process {file_path.name}: {e}")
        raise

df_str2 = pd.DataFrame(records)
output_csv_path = output_path / "str1.csv"
output_csv_path.parent.mkdir(parents=True, exist_ok=True)
df_str2.to_csv(output_csv_path, index=False)

df_str2


,filename,namespace,name,type,ObjektovyTypNazev_minOccurs,ObjektovyTypNazev,code_base_fixed,code_base_use,code_suffix_fixed,code_suffix_use,...,ObsahovaCast_minOccurs,ObsahovaCast,ZaznamyObjektu_minOccurs,ZaznamObjektu_minOccurs,AtributyObjektu_minOccurs,GeometrieObjektu_minOccurs,geom_minOccurs,geom,oblaskKI_minOccurs,OblastObjektuKI
0,BP_plynovodni_site-plocha.xsd,bpplsi,BPPlynovodniSite,bpplsi:BPPlynovodniSiteType,,bezpečnostní pásmo plynovodní sítě,0100000290,required,03,required,...,,TI,,,,0,0,gml:surfaceProperty,0,1.0
1,BP_podzemniho_zasobniku_plynu-plocha.xsd,bpppol,BPPodzemnihoZasobnikuPlynu,bpppol:BPPodzemnihoZasobnikuPlynuType,,bezpečnostní pásmo podzemního zásobníku plynu,0100000369,required,03,required,...,,TI,,,,0,0,gml:surfaceProperty,0,1.0
2,BP_zarizeni_PKO-plocha.xsd,bpzpko,BPZarizeniPKO,bpzpko:BPZarizeniPKOType,,bezpečnostní pásmo zařízení PKO,0100000291,required,03,required,...,,TI,,,,0,0,gml:surfaceProperty,0,1.0
3,budova-defbod.xsd,buddef,BudovaDefinicniBod,buddef:BudovaDefinicniBodType,,budova,0100000001,required,04,required,...,,ZPS,,,,,,gml:pointProperty,NaN,NaN
4,budova-plocha.xsd,budpol,BudovaPlocha,budpol:BudovaPlochaType,,budova,0100000001,required,03,required,...,,ZPS,,,,,,gml:surfaceProperty,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,zed-linie.xsd,zedlin,ZedLinie,zedlin:ZedLinieType,,zeď,0100000168,required,02,required,...,,ZPS,,,,,,gml:curveProperty,NaN,NaN
354,zed-plocha.xsd,zedpol,ZedPlocha,zedpol:ZedPlochaType,,zeď,0100000168,required,03,required,...,,ZPS,,,,,,gml:surfaceProperty,NaN,NaN
355,zeleznicni_prejezd-plocha.xsd,zprpol,ZeleznicniPrejezd,zprpol:ZeleznicniPrejezdType,,železniční přejezd,0100000022,required,03,required,...,,DI,,,,,,gml:surfaceProperty,NaN,NaN
356,zemedelska_plocha-defbod.xsd,zepdef,ZemedelskaPlochaDefinicniBod,zepdef:ZemedelskaPlochaDefinicniBodType,,zemědělská plocha,0100000207,required,04,required,...,,ZPS,,,,,,gml:pointProperty,NaN,NaN


In [10]:
from pathlib import Path
from lxml import etree
import pandas as pd

output_path = Path(r"C:\Users\michal\Desktop\FGIS\2025-b-jvf-dtm-xsd\tests\output\JVF_DTM_143_XSD")
xsd_objects_path = output_path / "xsd" / "objekty"

records = []

for file_path in xsd_objects_path.glob("*.xsd"):
    try:
        tree = etree.parse(str(file_path))
        root = tree.getroot()
        namespace = root.attrib.get("targetNamespace", "")

        record = {
            "filename": file_path.name,
            "namespace": namespace,
        }

        first_element = root.xpath(f".//*[local-name()='element'][starts-with(@type, '{namespace}:')]")
        if first_element:
            record["name"] = first_element[0].get("name", "")
            record["type"] = first_element[0].get("type", "")

        for complex_type in root.xpath(".//*[local-name()='complexType']"):
            for element in complex_type.xpath(".//*[local-name()='element']"):
                name = element.get("name") or element.get("ref") or "unknown_element"
                min_occurs = element.get("minOccurs", "")
                ref = element.get("ref")

                for attr_key, attr_value in element.attrib.items():
                    record[f"{name}__{attr_key}"] = attr_value

                attributes = element.xpath(".//*[local-name()='attribute']")
                for attr in attributes:
                    attr_name = attr.get("name", "unnamed")
                    for key, value in attr.attrib.items():
                        record[f"{name}__{attr_name}__{key}"] = value

                if name == "GeometrieObjektu":
                    ref_elem = element.xpath(".//*[local-name()='element' and @ref]")
                    if ref_elem:
                        record["geom"] = ref_elem[0].get("ref", "")

                if ref == "atr:OblastObjektuKI":
                    record["OblastObjektuKI"] = 1

        records.append(record)

    except etree.XMLSyntaxError as e:
        print(f"[XMLSyntaxError] {file_path.name}: {e}")
    except Exception as e:
        print(f"[Error] Failed to process {file_path.name}: {e}")
        raise

df_str2 = pd.DataFrame(records)
output_csv_path = output_path / "str1.csv"
output_csv_path.parent.mkdir(parents=True, exist_ok=True)
df_str2.to_csv(output_csv_path, index=False)

df_str2


,filename,namespace,name,type,ObjektovyTypNazev__name,ObjektovyTypNazev__fixed,ObjektovyTypNazev__code_base__name,ObjektovyTypNazev__code_base__type,ObjektovyTypNazev__code_base__fixed,ObjektovyTypNazev__code_base__use,...,atr:TypZarizeniKanalizacniPripojky__ref,atr:TypZarizeniProLeteckyProvoz__ref,atr:TypZarizeniPlynovodniSite__ref,atr:TypZarizeniPotrubniPosty__ref,atr:TypZarizeniPrivadece__ref,atr:TypZarizeniSiteEK__ref,atr:TypZarizeniSiteProduktovodu__ref,atr:TypZarizeniTeplovodniSite__ref,atr:TypZarizeniVodovodniPripojky__ref,atr:TypZemedelskePlochy__ref
0,BP_plynovodni_site-plocha.xsd,bpplsi,BPPlynovodniSite,bpplsi:BPPlynovodniSiteType,ObjektovyTypNazev,bezpečnostní pásmo plynovodní sítě,code_base,xs:string,0100000290,required,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BP_podzemniho_zasobniku_plynu-plocha.xsd,bpppol,BPPodzemnihoZasobnikuPlynu,bpppol:BPPodzemnihoZasobnikuPlynuType,ObjektovyTypNazev,bezpečnostní pásmo podzemního zásobníku plynu,code_base,xs:string,0100000369,required,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BP_zarizeni_PKO-plocha.xsd,bpzpko,BPZarizeniPKO,bpzpko:BPZarizeniPKOType,ObjektovyTypNazev,bezpečnostní pásmo zařízení PKO,code_base,xs:string,0100000291,required,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,budova-defbod.xsd,buddef,BudovaDefinicniBod,buddef:BudovaDefinicniBodType,ObjektovyTypNazev,budova,code_base,xs:string,0100000001,required,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,budova-plocha.xsd,budpol,BudovaPlocha,budpol:BudovaPlochaType,ObjektovyTypNazev,budova,code_base,xs:string,0100000001,required,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,zed-linie.xsd,zedlin,ZedLinie,zedlin:ZedLinieType,ObjektovyTypNazev,zeď,code_base,xs:string,0100000168,required,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354,zed-plocha.xsd,zedpol,ZedPlocha,zedpol:ZedPlochaType,ObjektovyTypNazev,zeď,code_base,xs:string,0100000168,required,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355,zeleznicni_prejezd-plocha.xsd,zprpol,ZeleznicniPrejezd,zprpol:ZeleznicniPrejezdType,ObjektovyTypNazev,železniční přejezd,code_base,xs:string,0100000022,required,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,zemedelska_plocha-defbod.xsd,zepdef,ZemedelskaPlochaDefinicniBod,zepdef:ZemedelskaPlochaDefinicniBodType,ObjektovyTypNazev,zemědělská plocha,code_base,xs:string,0100000207,required,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,atr:TypZemedelskePlochy
